<a href="https://colab.research.google.com/github/NH0917/ghg/blob/main/autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogluon
!pip install dask[dataframe]

In [ ]:
#!cat /proc/cpuinfo

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
#import optuna.integration.lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_log_error,mean_squared_error
#from openfe import OpenFE, transform
import io
import contextlib
#from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.preprocessing import LabelEncoder
import warnings

from autogluon.tabular import TabularDataset
from autogluon.tabular import TabularPredictor

from logging import getLogger, INFO

logger = getLogger(__name__)
logger.setLevel("WARNING")

warnings.simplefilter('ignore')

import sys

sys.path.append("/content/drive/MyDrive/signate/ghg/src/")
pd.set_option("display.max_columns",100)

#import my code
#import sub
#import plot_importance
#import train_evall_lgb

In [ ]:
class cfg:
    """Configuration class that holds hyperparameters for quicker experimenting"""
    num_folds = 10
    random_seed = 1
    n_jobs = 16
    n_features = 10

In [ ]:
train_path = "/content/drive/MyDrive/signate/ghg/exp_new/exp3/train_df_openfe_v2.csv"
test_path = "/content/drive/MyDrive/signate/ghg/exp_new/exp3/test_df_openfe_v2.csv"
sub_path = "/content/drive/MyDrive/signate/ghg/data/sample_submission.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sub_df = pd.read_csv(sub_path,header=None)

In [ ]:
def get_rmsle(y_true: np.array, y_pred: np.array) -> np.float64:
    """
    The Root Mean Squared Log Error (RMSLE) metric

    :param y_true: The ground truth labels given in the dataset
    :param y_pred: Our predictions
    :return: The RMSLE score
    """

    return mean_squared_log_error(y_true, y_pred, squared=False)

In [ ]:
target_col = "GHG_Direct_Emissions_14_in_metric_tons"

In [ ]:
openfe_col = [x for x in train.columns if "autoFE_f" in x]
openfe_col = openfe_col[:20]
ghg_col = [x for x in train.columns if "GHG_" in x]

In [ ]:
usecol = openfe_col + ghg_col

In [ ]:
train = train[usecol]
test = test[usecol]

In [ ]:
mean_col = ['GHG_Direct_Emissions_10_in_metric_tons',
 'GHG_Direct_Emissions_11_in_metric_tons',
 'GHG_Direct_Emissions_12_in_metric_tons',
 'GHG_Direct_Emissions_13_in_metric_tons']

train["GHG_mean"] = train[mean_col].mean(axis=1)

In [ ]:
def rmsle(y_true, y_pred):
    rmsle = mean_squared_error(y_true, y_pred)
    return rmsle


from sklearn.model_selection import KFold

models = []


N_FOLDS = 5
TIMEOUT = 300
N_THREADS = 8

oof_dfs = pd.DataFrame()

kf = KFold(n_splits=cfg.num_folds,shuffle=True,random_state=42)


train[target_col] = np.log1p(train[target_col])

for i,(train_idx,val_idx) in enumerate(kf.split(train)):
  train_df = train.iloc[train_idx].reset_index(drop=True)
  val_df = train.iloc[val_idx].reset_index(drop=True)

  oof_df = pd.DataFrame()

  train_df = TabularDataset(train_df)
  val_df = TabularDataset(val_df)

  automl = TabularPredictor(label=target_col,eval_metric="rmse",verbosity=0).fit(train_df,time_limit= 300,presets='best_quality')

  oof_df["target"] = val_df[target_col]
  oof_df["predict"] = automl.predict(val_df)
  print(rmsle(oof_df["target"],oof_df["predict"]))

  models.append(automl)

  oof_dfs = pd.concat([oof_dfs,oof_df]).reset_index(drop=True)

0.4901333151351153
0.6047807055844813
0.48170404100640873
0.634832349290872
0.5579045574099587
0.6690525185319344
0.6043530066960796
0.5615334410855655
0.559146464620743
0.580441356828492


In [ ]:
oof_dfs.reset_index(drop=True,inplace=True)

In [ ]:
oof_dfs["target"] = np.expm1(oof_dfs["target"])
oof_dfs["predict"] = np.expm1(oof_dfs["predict"])

In [ ]:
from sklearn.metrics import mean_squared_error

def rmsle(y_true, y_pred):
    rmsle = mean_squared_error(y_true, y_pred)
    return rmsle

rmsle(oof_dfs["target"],oof_dfs["predict"])

In [ ]:
get_rmsle(oof_dfs["target"],oof_dfs["predict"])

0.7578694728180854

In [ ]:
for ml in models:
  display(ml.leaderboard())

In [ ]:
import matplotlib.pyplot as plt
for ml in models:
  fast_fi = ml.get_feature_scores("fast")
  fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)
  plt.show()

In [ ]:
import seaborn as sns

sns.scatterplot(x="GHG_mean",y=target_col,data=train)

In [ ]:
pred_df = pd.DataFrame()

fold = 1
for ml in models:
  pred_df[f"{fold}"] = ml.predict(test)
  pred_df[f"{fold}"] = np.expm1(pred_df[f"{fold}"])
  fold += 1

In [ ]:
sub_df[1] = pred_df.mean(axis=1)

In [ ]:
output_path = "/content/drive/MyDrive/signate/ghg/exp_new/exp4"

sub_df.to_csv(f"{output_path}/sub.csv",index=False)

In [ ]:
oof_dfs.to_csv(f"{output_path}/oof.csv",index=False)

In [ ]:
oof_dfs

,target,predict
0,8.017493e+04,7.605124e+04
1,6.116077e+05,7.910235e+05
2,1.090341e+06,4.503138e+05
3,2.323973e+05,1.320049e+05
4,1.491725e+04,3.440918e+04
...,...,...
4650,4.097412e+04,3.347876e+04
4651,7.437145e+04,8.005576e+04
4652,4.067482e+04,4.413943e+04
4653,1.600409e+06,1.207200e+06
